In [152]:
%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [153]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from enum import IntEnum
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import trange
from collections import defaultdict

from GrU_nn import (gru_module_1, gru_module_2, gru_module_3, gru_module_4, neuralGrU)
from GrU import (evaluate, GrU, getBestExporter, getCost)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

### Load auction data

In [154]:
class clock( IntEnum ):
    pv = 0
    storage = 1
    charge = 2
    discharge = 3

class source( IntEnum ):
    grid =  0
    pv = 1
    storage = 2
    charge = 3
    discharge = 4

df_auctioneer_data = pd.read_csv('../data/Auctioneer Data.csv')

headers = np.load(file='../data/clock_data_columns.npy', allow_pickle=True)
df_clock_data = pd.DataFrame(np.load('../data/clock_data_values.npy', allow_pickle=True), columns=headers)

def getPriceVectors(round):
    '''
    Get price vectors for a round from the clock data.

    Parameters
    ----------
    round : int
        Round number to get the price vectors for, range = [0, 308]

    Returns
    -------
    prices : ndarray of shape (5, 24)
        Price vectors for g, p, s, c, d.
    
    '''
    prices = np.ndarray((5, 24))
    prices[0] = df_auctioneer_data.GRID_PRICE.values
    for e in clock:
        prices[e + 1] = df_clock_data[f'price_{round}'][e]
    return prices

## Neural GrU

### Experiment -> Verify the output of Module-1

In [165]:
prices = getPriceVectors(100) * 1e2
eta = 1
n_agents = 1

pi_tilda_gru = []
i_t_gru = []

for t in range(24):
    pi, i = getBestExporter(prices, t, eta, eta)
    pi_tilda_gru.append(pi)
    i_t_gru.append(i)

m1 = gru_module_1(n_agents, device, eta, eta)

pi_s = torch.tensor(prices[source.storage]).expand((n_agents, -1))
pi_c = torch.tensor(prices[source.charge]).expand((n_agents, -1))
pi_g = torch.tensor(prices[source.grid]).expand((n_agents, -1))

pi_tilda_nn, i_t_nn = m1(pi_s, pi_c, pi_g, B=1e2)

In [166]:
i_t_gru

[-1, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

In [169]:
i_t_nn.shape

torch.Size([1, 24, 24])

In [170]:
torch.argmax(i_t_nn, dim=2)

tensor([[0, 0, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]])

In [161]:
i_t_gru - torch.argmax(i_t_nn, dim=1).numpy()

array([[-2, -2, -2, -2, -3, -4, -5,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2]])

In [162]:
pi_tilda_nn - torch.tensor(pi_tilda_gru)

tensor([[        nan, -1.2158e-06, -4.4983e-07,  5.6641e-07, -6.5430e-07,
          9.4788e-07,  7.1899e-07,  4.1382e-07,  1.0864e-07, -1.9653e-07,
         -5.0171e-07, -8.0688e-07,  7.9529e-07,  4.9011e-07,  1.8494e-07,
         -1.2024e-07,  4.1382e-07, -1.2024e-07,  4.1382e-07, -1.2024e-07,
          7.1899e-07, -4.3945e-08, -8.0688e-07,  3.3752e-07]],
       dtype=torch.float64, grad_fn=<SubBackward0>)

### Experiment -> Verify the output of Module-1 v2

In [171]:
n_agents = 1
m2 = gru_module_2(eta, alpha=1e3)

In [174]:
pi_d = torch.tensor(prices[source.discharge]).expand((n_agents, -1))
pi_p = torch.tensor(prices[source.pv]).expand((n_agents, -1))

a_1_t, a_2_t, a_3_t = m2(pi_tilda_nn, pi_d, pi_p, pi_g, B=1e2)

In [175]:
a_1_t.shape

torch.Size([1, 24, 3])

In [176]:
torch.round(a_1_t[0], decimals=3)[:6]

tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.]], dtype=torch.float64, grad_fn=<SliceBackward0>)

In [177]:
torch.round(a_2_t[0], decimals=3)[:6]

tensor([[0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.]], dtype=torch.float64, grad_fn=<SliceBackward0>)

In [178]:
torch.round(a_3_t[0], decimals=3)[:6]

tensor([[0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.]], dtype=torch.float64, grad_fn=<SliceBackward0>)

### Experiment -> Forward pass of Neural GrU

In [196]:
eta = 1
B = 1e3
alpha = 1e3
n_agents = 1
ep = np.ones(24)

m = neuralGrU(B, n_agents, ep, [0], device, eta, eta)

In [191]:
prices = getPriceVectors(300) * 100
Caps = np.ones((1, 24, 3)) * 1e2
Caps[:, :, 0] = 4  # Discharge cap
Caps[:, :, 1] = 3  # PV cap
d_total = np.ones((1, 24)) * 5

# C_t = torch.tensor(Caps)
pi = torch.tensor(prices)
d_t = torch.tensor(d_total)

d_break, _ = GrU(d_total[0], prices, eta_c=eta, eta_d=eta)
d_break_nGrU = m(pi[0].expand((1, -1)), pi[1].expand((1, -1)), pi[2].expand((1, -1)), pi[3].expand((1, -1)), pi[4].expand((1, -1)), d_t, ).detach().numpy()

In [192]:
d_break_nGrU.round(1)

array([[[ 5.,  5.,  5.,  5.,  5.,  5.,  5., 20.,  0.,  0.,  0.,  0.,
          5.,  5.,  5.,  5.,  5., 35.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  5.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0., 15., 10.,  5.,  0.,  0.,
          0.,  0.,  0.,  0.,  0., 30., 25., 20., 15., 10.,  5.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0., 15.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0., 30.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  5.,  5.,  5.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  5.,  5.,  5.,  5.,  5.,  5.]]])

In [193]:
np.round(evaluate(d_break, d_break_nGrU[0])[0], decimals=3)

array([0., 0., 0., 0., 0.])

In [194]:
d_break.round(2)

array([[ 5.,  5.,  5.,  5.,  5.,  5.,  5., 20.,  0.,  0.,  0.,  0.,  5.,
         5.,  5.,  5.,  5., 35.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  5.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., 15., 10.,  5.,  0.,  0.,  0.,
         0.,  0.,  0.,  0., 30., 25., 20., 15., 10.,  5.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., 15.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0., 30.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  5.,  5.,  5.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  5.,  5.,  5.,  5.,  5.,  5.]])

### Experiment -> Verify the output(s) of Neural GrU Modules with toy example

> Outdated code; needs to be updated.

In [15]:
B = 1e3
eta = 1

m1 = gru_module_1(device, eta, eta)
m2 = gru_module_2(eta)
m3 = gru_module_3()
m4 = gru_module_4(eta, eta)

pi_g = torch.tensor([2, 4, 8, 1] + [1] * 20, dtype=float)
pi_p = torch.tensor([1, 2, 4, 2] + [2] * 20, dtype=float)
pi_c = torch.ones(24, dtype=float)
pi_d = torch.ones(24, dtype=float)
pi_s = torch.ones(24, dtype=float)

d_t = torch.ones(24)

pi_tilda_c_t, i_t = m1(pi_s, pi_c, pi_g, B)
a_t = m2(pi_tilda_c_t, pi_d, pi_p, pi_g, B)
i_tilda_t = m3(i_t)
d_star = m4(d_t, i_t, a_t, i_tilda_t)

In [16]:
print('grid prices =',pi_g[:5])
print('pv prices =',pi_p[:5])
print('storage prices =',pi_s[:5])
print('charging prices =',pi_c[:5])
print('discharging prices =',pi_d[:5])

grid prices = tensor([2., 4., 8., 1., 1.], dtype=torch.float64)
pv prices = tensor([1., 2., 4., 2., 2.], dtype=torch.float64)
storage prices = tensor([1., 1., 1., 1., 1.], dtype=torch.float64)
charging prices = tensor([1., 1., 1., 1., 1.], dtype=torch.float64)
discharging prices = tensor([1., 1., 1., 1., 1.], dtype=torch.float64)


> Output of Module-1: Price of importing power from cheapest time slot

In [17]:
pi_tilda_c_t[:6]

tensor([inf, 4., 5., 6., 3., 3.], grad_fn=<SliceBackward0>)

In [18]:
torch.argmax(i_t[:6], dim=1)

tensor([0, 0, 0, 0, 3, 4])

> Output of Module-2: Cheapest source b/w discharge(incuding import from cheapest time slot), pv & grid

In [19]:
a_t[:6]

tensor([[0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.]], dtype=torch.float64, grad_fn=<SliceBackward0>)

> Output of Module-3: Time slots (one hot vector) to store the charge to be discharged at time slot t

In [20]:
i_tilda_t[:6]

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]], grad_fn=<SliceBackward0>)

> Output of Module-4

In [21]:
d_star[:, :6]

tensor([[0., 0., 0., 1., 1., 1.],
        [1., 1., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]], dtype=torch.float64,
       grad_fn=<SliceBackward0>)